### 1. Загрузка данных

In [1]:
!pip install geopandas

  Using cached geopandas-0.10.2-py2.py3-none-any.whl (1.0 MB)
  Using cached pyproj-3.3.1-cp38-cp38-win_amd64.whl (6.4 MB)
  Using cached Fiona-1.8.21.tar.gz (1.0 MB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'error'


  error: subprocess-exited-with-error
  
  python setup.py egg_info did not run successfully.
  exit code: 1
  
  [1 lines of output]
  A GDAL API version must be specified. Provide a path to gdal-config using a GDAL_CONFIG environment variable or use a GDAL_VERSION environment variable.
  [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
error: metadata-generation-failed

Encountered error while generating package metadata.

See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [3]:
import pandas as pd
import numpy as np
from zipfile import ZipFile
import fnmatch
from tqdm.notebook import tqdm
from rosreestr2coord import Area #https://github.com/rendrom/rosreestr2coord
import folium
from shapely.geometry import Polygon

In [4]:
pd.set_option('display.max_columns', 25)
#pd.reset_option("display.max_columns")

In [5]:
pd.set_option("max_colwidth", 400)

Заходи на сайт Федеральной службы государственной регистрации, кадастра и картографии и в разделе "Открытая служба" скачиваем дата-сет "Данные о кадастровой стоимости объектов недвижимости в разрезе территориальной принадлежности" для Свердловской области. https://rosreestr.gov.ru/open-service/data-sety-rosreestra/dannye-o-kadastrovoy-stoimosti-obektov-nedvizhimosti-v-razreze-territorialnoy-prinadlezhnosti/

Так как архив состоит из множества json файлов, поэтому в цикле соберем их в один датафрейм, но на итеррациях будем отбирать данные только по Екатеринбургу

In [6]:
path = 'D:\\4. Программы\\price_66\\export-json-region-66-date-1639121273.zip'

In [7]:
with ZipFile(path) as zipfiles:
    
    df_ekb_total = pd.DataFrame()
    file_list = zipfiles.namelist() # получим список файлов в архиве  
    
    json_files = fnmatch.filter(file_list, "flat_ds1*.json") # отбирем только файлы с квартирами
    n = len(json_files)//100 + 1 
    df = pd.DataFrame()

    for i in tqdm(range(n+1)):
        
        for json_file in json_files[i*n: (i+1)*n]:    
   
            data = pd.read_json(zipfiles.open(json_file))
    
            # столбцы 'purpose', 'cadCost', 'address' содержат json, поэтому разобьём их на колонки
            data_purpose = pd.json_normalize(data['purpose'])
            data_cost = pd.json_normalize(data['cadCost'])
            data_address = pd.json_normalize(data['address'])
            data = data.drop(columns=['purpose', 'cadCost', 'address'])
            data = pd.concat([data, data_purpose, data_cost, data_address],axis=1)
            
            data = data[(data['city']=='Екатеринбург')|(data['locality']=='Екатеринбург')]
            df_ekb_total = pd.concat([df_ekb_total, data],axis=0, ignore_index=True)

In [8]:
df_ekb_total.to_csv('D:\\4. Программы\\price_66\\ekb.csv', index=False)

In [9]:
df_ekb_total.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 298012 entries, 0 to 298011
Data columns (total 23 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   cadBlockNum        298012 non-null  object 
 1   type               298012 non-null  object 
 2   status             298012 non-null  int64  
 3   objKind            298012 non-null  int64  
 4   area               298012 non-null  float64
 5   code               297316 non-null  object 
 6   text               298010 non-null  object 
 7   costValue          298012 non-null  float64
 8   upks               298012 non-null  float64
 9   registrationDate   298012 non-null  object 
 10  determinationDate  298012 non-null  object 
 11  regionCode         298012 non-null  object 
 12  region             298012 non-null  object 
 13  city               298012 non-null  object 
 14  cityType           298012 non-null  object 
 15  street             296128 non-null  object 
 16  st

In [10]:
df_ekb_total.head(3)

,cadBlockNum,type,status,objKind,area,code,text,costValue,upks,registrationDate,determinationDate,regionCode,region,city,cityType,street,streetType,locality,localityType,district,districtType,urbanDistrict,urbanType
0,66:41:0502074,flat,1,2001003000,37.3,206002000000,жилое помещение,2813133.30,75419.12,2020-01-09,2019-01-01,66,Свердловская область,Екатеринбург,г,Павлодарская,ул,NaN,NaN,NaN,NaN,NaN,NaN
1,66:41:0704008,flat,1,2001003000,39.4,206002000000,жилое помещение,3346813.65,84944.50,2020-01-11,2019-01-01,66,Свердловская область,Екатеринбург,г,Блюхера,ул,NaN,NaN,NaN,NaN,NaN,NaN
2,66:41:0509048,flat,1,2001003000,47.3,206002000000,жилое помещение,2806336.44,59330.58,2020-01-12,2019-01-01,66,Свердловская область,Екатеринбург,г,Черняховского,ул,NaN,NaN,NaN,NaN,NaN,NaN


Сделаем сводник по кварталам ипосчитаем основные метрики

In [15]:
df_ekb = df_ekb_total.groupby('cadBlockNum').agg(cost_value_mean = ('costValue', 'mean'),
                                        cost_value_median = ('costValue', 'median'),
                                        upks_mean = ('upks', 'mean'),
                                        upks_median = ('upks', 'median'),
                                        area_mean = ('area', 'median'),
                                        area_median = ('area', 'median')
                                        ).reset_index()

In [16]:
df_ekb.head()

,cadBlockNum,cost_value_mean,cost_value_median,upks_mean,upks_median,area_mean,area_median
0,66:40:0101021,1.322361e+05,132236.13,5556.140000,5556.140,23.80,23.80
1,66:41:0000000,3.440285e+06,2684789.31,52532.467541,60346.270,39.80,39.80
2,66:41:0101001,1.406866e+06,1406865.90,9662.540000,9662.540,145.60,145.60
3,66:41:0102006,1.584646e+06,1841561.67,46385.383750,47147.475,38.15,38.15
4,66:41:0102009,6.218820e+05,621881.97,12412.810000,12412.810,50.10,50.10


Для номеров кадастровых кварталов вычислим координаты с помощью функции rosreestr2coord.Area (https://github.com/rendrom/rosreestr2coord) и перобразуем их в полигоны

In [17]:
kvartal_list = df_ekb.cadBlockNum.to_list()

In [18]:
area_koord_list = []
for kvartal in tqdm(kvartal_list):
    area = Area(code=kvartal, area_type=2, with_log=False, with_proxy=True)
    area_koord = area.get_coord()
    area_koord_list.append(area_koord)

In [19]:
df_ekb['kvartal'] = area_koord_list

In [20]:
df_ekb['kvartal'] = df_ekb['kvartal'].apply(lambda x: x[0][0])

In [21]:
# оставим в датафрейме строки в которых количество координатных пар в колонке kvartal больше двух
df_ekb = df_ekb[df_ekb['kvartal'].apply(lambda x: len(x)>2)]

In [22]:
df_ekb['kvartal'] = df_ekb['kvartal'].apply(lambda x: Polygon(x)) 

Сохраним на комп полученную таблицу в формате .csv и построим по ней в Tableau дашборд

In [23]:
df_ekb.to_csv('D:\\4. Программы\\price_66\\ekb.csv', index=False)